In [150]:
import pickle
import os
import re

from glob import glob
from gensim import models
from nltk.corpus import stopwords
from FlaskApp.transform import *
from googletrans import Translator
from string import punctuation
from googletrans import Translator

import pandas as pd
import numpy as np

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

translator = Translator()

In [151]:
filepaths = glob('data/scraped/jobstreet*.pickle')

In [152]:
all_jobs = []

for filepath in filepaths:
    with open(filepath, 'rb') as file:
        loaded = pickle.load(file)
        
        for job in loaded:
            all_jobs.append(job)

In [158]:
df_job = pd.DataFrame({
    'Title': [job['Title'] for job in all_jobs],
    'Company': [job['Company'] for job in all_jobs],
    'Area': [job['Area'] for job in all_jobs],
    'Description': [job['Description'] for job in all_jobs],
    'Information': [job['Information'] for job in all_jobs],
})

df_job.dropna(subset=['Description', 'Title'], inplace=True)

df_job.Description = df_job.Description.apply(lambda x: ' \n '.join(x) if x != None else '')
df_job.Information = df_job.Information.apply(lambda x: ' \n '.join(x) if x != None else '')
# df_job.Description = df_job.Description.apply(lambda x: ' \\n '.join(x) if x != None else '')
# df_job.Description = df_job.Description.map(str.lower)

# # menghapus karakter selain punctuation, word, dan space
# df_job.Description = df_job.Description.apply(lambda x: re.sub(r'[^\w\s!"#$%&\'()*+,-./:;<=>?@[\\\]^_`{|}~]', '', x))

# # menambahkan spasi di setiap symbol
# # df_job.Description = df_job.Description.apply(lambda x: re.sub(r'(?<=\W)([.])(?=\W)', r' \1 ', x))
# df_job.Description = df_job.Description.apply(lambda x: x.replace('.', ' . '))

# df_job.Description = df_job.Description.map(remove_morespace)

# df_job.Information = df_job.Information.apply(lambda x: ' '.join(x) if x != None else '')

In [159]:
df_job.drop_duplicates(inplace=True)

In [169]:
all_jobstreet = []

for i in df_job.index:
    job = {
        'Title': df_job.Title.loc[i],
        'Company': df_job.Company.loc[i],
        'Area': df_job.Area.loc[i],
        'Description': df_job.Description.loc[i],
        'Information': df_job.Information.loc[i],
    }
    all_jobstreet.append(job)

In [171]:
with open('data/scraped/all_jobstreet.pickle', 'wb') as file:
    pickle.dump(all_jobstreet, file)

with open('data/scraped/all_jobstreet.pickle', 'rb') as file:
    loaded = pickle.load(file)

Translating

In [ ]:
all_jobstreet = []

In [177]:
for i, job in enumerate(all_jobs):
    if job['Title'] == None or job['Description'] == None:
        continue
    if 'Language' not in job.keys():
        desc = ' '.join(job['Description'])
        lang = translator.detect(desc).lang

        job['Language'] = lang

        print(i, '✅', job['Title'], lang, 'language detected!')
else:
    print('Loop ending 👍 ..')

1331 ✅ Accounting Staff (Project Based) en language detected!
1332 ✅ Admin Purchasing id language detected!
1333 ✅ Staf Finance id language detected!
1334 ✅ Finance & System Officer id language detected!
1335 ✅ FINANCE OFFICER (AP) id language detected!
1336 ✅ STAFF ACCOUNTING id language detected!
1337 ✅ Administration Staff en language detected!
1338 ✅ Accounting Staff en language detected!
1339 ✅ Junior Staff en language detected!
1340 ✅ Accounting id language detected!
1341 ✅ Admin Account Payable id language detected!
1342 ✅ ACCOUNTING FINANCE id language detected!
1343 ✅ Finance AR Staff en language detected!
1344 ✅ STAFF ACCOUNTING DAN FINANCE id language detected!
1345 ✅ Staff Accounting id language detected!
1346 ✅ Finance & Accounting Staff en language detected!
1347 ✅ Accounting Officer id language detected!
1348 ✅ Staff Accounting id language detected!
1349 ✅ Staff Accounting id language detected!
1350 ✅ Staff Accounting id language detected!
1351 ✅ Accounting General Ledge

ConnectTimeout: timed out

In [179]:
all_jobstreet

[]

Processing Model

In [90]:
df_job.isna().sum()

Title          0
Company        0
Area           0
Description    0
Information    4
dtype: int64

In [92]:
df_job.head(1)

,Title,Company,Area,Description,Information
0,CAD Drafter,PT Indo Plafon,Tangerang,"Responsibility: Prepare drawings, detailing, creative design works and technical documents Doing area, quantity calculation Site survey Project site supervision, coordination Requirement: Must possess at least a Diploma in Architectural or Interior Design Preferably with minimum 3-5 years of working experience in architectural / building interior industry or relevant field. Well-versed in operating AutoCAD, Photoshop, 3D Sketch Up Willing to travel outstation/ overseas when required Good written and oral communication skills in Bahasa Indonesia and English","[Prepare drawings, detailing, creative design works and technical documents, Site survey, Requirement:, Preferably with minimum 3-5 years of working experience in architectural / building interior industry or relevant field., Willing to travel outstation/ overseas when required]"


In [109]:
dataset = df_job.Description.map(str.split).values

data = []
for word in dataset:
    data.append(word)

def tagged_document(list_of_listwords, tags=None):
    if tags is not None:
        for x, listwords in zip(tags, list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])
    else:
        for x, listwords in enumerate(list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])

data_train = list(tagged_document(data, df_job.Title.values))


In [110]:
d2v_model = models.doc2vec.Doc2Vec(vector_size=100, min_count=10, epochs=50, window=5)
d2v_model.build_vocab(data_train)

2023-05-05 15:47:20,620 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d100,n5,w5,mc10,s0.001,t3>', 'datetime': '2023-05-05T15:47:20.620401', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-05-05 15:47:20,622 : INFO : collecting all words and their counts
2023-05-05 15:47:20,623 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-05-05 15:47:21,214 : INFO : PROGRESS: at example #10000, processed 1697574 words (2879794 words/s), 64441 word types, 5183 tags
2023-05-05 15:47:21,568 : INFO : PROGRESS: at example #20000, processed 3346066 words (4650005 words/s), 93510 word types, 10186 tags
2023-05-05 15:47:21,901 : INFO : PROGRESS: at example #30000, processed 4941327 words (4817116 words/s), 110939 word types, 13581 tags
2023-05-05 15:47:22,188 : INFO : PROGRESS: at example #40000, processed 640

In [111]:
list(d2v_model.wv.index_to_key)

['\\n',
 'and',
 'dan',
 'to',
 'the',
 'in',
 'yang',
 'of',
 'dengan',
 'with',
 '-',
 'a',
 'di',
 'for',
 '&',
 'untuk',
 'Memiliki',
 'tahun',
 ':',
 '•',
 'bekerja',
 'kerja',
 'or',
 'dalam',
 'baik',
 'minimal',
 'pengalaman',
 'as',
 'is',
 'experience',
 '/',
 'Melakukan',
 'work',
 'team',
 'on',
 'are',
 'Mampu',
 'memiliki',
 'all',
 '1',
 'pada',
 'be',
 'atau',
 'Pendidikan',
 'customer',
 'sales',
 'hari',
 'sesuai',
 'Membuat',
 'years',
 'jawab',
 'that',
 'posisi',
 'bisa',
 'Kualifikasi',
 'secara',
 'kemampuan',
 'Job',
 'perusahaan',
 'our',
 'at',
 'Persyaratan',
 'Usia',
 'bidang',
 'ini',
 'skills',
 'Deskripsi',
 'dari',
 'business',
 'an',
 'target',
 'from',
 'related',
 'will',
 'by',
 'Mencari',
 'laporan',
 'communication',
 '2',
 'data',
 'harus',
 'marketing',
 'produk',
 'sebagai',
 'other',
 'working',
 'serta',
 'management',
 'Minimal',
 'penjualan',
 'tim',
 'minimum:',
 'Bersedia',
 'media',
 'KTP',
 'Pelamar',
 'S1',
 'Bertanggung',
 'Pengalaman'

In [112]:
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-05-05 15:47:41,712 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 28064 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2023-05-05T15:47:41.712078', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-05-05 15:47:42,729 : INFO : EPOCH 0 - PROGRESS: at 20.62% examples, 1209912 words/s, in_qsize 6, out_qsize 0
2023-05-05 15:47:43,731 : INFO : EPOCH 0 - PROGRESS: at 42.84% examples, 1221855 words/s, in_qsize 6, out_qsize 0
2023-05-05 15:47:44,739 : INFO : EPOCH 0 - PROGRESS: at 65.49% examples, 1217554 words/s, in_qsize 5, out_qsize 0
2023-05-05 15:47:45,749 : INFO : EPOCH 0 - PROGRESS: at 89.35% examples, 1215514 words/s, in_qsize 5, out_qsize 0
2023-05-05 15:47:46,078 : INFO : EPOCH 0: training on 6772175 raw words (5260073 effective words) took 4.4s, 1

In [113]:
jobstreet_words = d2v_model.wv.index_to_key
jobstreet_vectors = d2v_model.wv.vectors

with open('data/jobstreet_vectors.tsv', 'w', encoding='utf-8') as f:
    for i, word in enumerate(jobstreet_words):
        vector_str = '\t'.join([str(num) for num in jobstreet_vectors[i]])
        f.write(f'{vector_str}\n')

with open('data/jobstreet_metadata.tsv', 'w', encoding='utf-8') as f:
    for word in jobstreet_words:
        f.write(f'{word}\n')